In [1]:
import pandas as pd
import numpy as np
import os
import requests
import env
import john_acquire as a

In [48]:
%load_ext autoreload
%autoreload 2

# Set the option to display all columns
pd.set_option('display.max_columns', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
# Example usage:
app_token = env.app_token
year_to_retrieve = '2023'
max_req = 100  # Specify the maximum number of observations to retrieve


In [61]:
inspections_df = a.get_health_inspection_data(year_to_retrieve, app_token)
inspections_df.head(5)

CSV file for 2023 already exists. Loading data from the CSV.


,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,violation_code,violation_description,critical_flag,score,grade,grade_date,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta
0,50124301,YUMMY JUICE BAR,Bronx,737A,LYDIG AVENUE,10462.0,3472936151,"Juice, Smoothies, Fruit Salads",2023-01-03T00:00:00.000,Violations were cited in the following area(s).,10B,Anti-siphonage or back-flow prevention device ...,Not Critical,12.0,A,2023-01-03T00:00:00.000,2023-10-24T06:00:07.000,Pre-permit (Operational) / Re-inspection,40.854471,-73.866174,211.0,13.0,22403.0,2049410.0,2.043190e+09,BX49
1,50106430,PAN TODO RICO,Queens,7617,ROOSEVELT AVE,11372.0,6466393116,Bakery Products/Desserts,2023-01-03T00:00:00.000,Violations were cited in the following area(s).,04H,"Raw, cooked or prepared food is adulterated, c...",Critical,32.0,C,2023-01-03T00:00:00.000,2023-10-24T06:00:07.000,Cycle Inspection / Re-inspection,40.747064,-73.889337,403.0,25.0,28700.0,4029880.0,4.012870e+09,QN28
2,50129821,MEI JUNG MEI CHINESE RESTAURANT,Brooklyn,1402,FLATBUSH AVENUE,11210.0,9292509943,Chinese,2023-01-03T00:00:00.000,Violations were cited in the following area(s).,04L,Evidence of mice or live mice in establishment...,Critical,18.0,B,2023-01-03T00:00:00.000,2023-10-24T06:00:07.000,Pre-permit (Operational) / Re-inspection,40.636380,-73.951435,314.0,45.0,77000.0,3120854.0,3.052260e+09,BK42
3,50057824,SERAFINA LUDLOW,Manhattan,98,RIVINGTON STREET,10002.0,2123589800,Italian,2023-01-03T00:00:00.000,Violations were cited in the following area(s).,08A,Establishment is not free of harborage or cond...,Not Critical,19.0,N,NaN,2023-10-24T06:00:07.000,Cycle Inspection / Initial Inspection,40.720111,-73.988463,103.0,1.0,3001.0,1084639.0,1.004110e+09,MN27
4,41399360,NEW RONG HANG RESTAURANT,Manhattan,38,ELDRIDGE STREET,10002.0,2126258999,Chinese,2023-01-03T00:00:00.000,Violations were cited in the following area(s).,04M,Live roaches in facility's food or non-food area.,Critical,27.0,B,2023-01-03T00:00:00.000,2023-10-24T06:00:07.000,Cycle Inspection / Re-inspection,40.715711,-73.993204,103.0,1.0,1600.0,1003876.0,1.003000e+09,MN27


In [53]:
inspections_df.head(5).to_csv('inspections_df.csv', index=False)

In [47]:
inspections_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73512 entries, 0 to 73511
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   camis                  73512 non-null  int64  
 1   dba                    73508 non-null  object 
 2   boro                   73512 non-null  object 
 3   building               73454 non-null  object 
 4   street                 73512 non-null  object 
 5   zipcode                72777 non-null  float64
 6   phone                  73512 non-null  object 
 7   cuisine_description    73512 non-null  object 
 8   inspection_date        73512 non-null  object 
 9   action                 73512 non-null  object 
 10  violation_code         73127 non-null  object 
 11  violation_description  73127 non-null  object 
 12  critical_flag          73512 non-null  object 
 13  score                  70147 non-null  float64
 14  grade                  41234 non-null  object 
 15  gr

In [3]:
g_places_api_key = env.g_places_api_key

In [125]:
import pandas as pd
import json
import requests

# Initialize empty lists to store the results for places, reviews, and API logs
places_results = []
reviews_results = []
api_logs = []

# Define the URL and headers
url = 'https://places.googleapis.com/v1/places:searchText'
headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': g_places_api_key,
    'X-Goog-FieldMask': 'places.id,places.displayName,places.formattedAddress,places.reviews'
}

# Parameters for saving progress
save_interval = 10  # Save after every 10 rows
current_row = 0

# Iterate over each row of the inspections_df DataFrame
for index, row in inspections_df.iloc[39:40].iterrows():
    current_row += 1
    # Construct the text query using dba, building, street, borough, and zipcode
    text_query = f"{row['dba']} {row['building']} {row['street']} {row['boro']}"
    if pd.notnull(row['zipcode']):  # Check if the zipcode is not null
        text_query += f" {int(row['zipcode'])}"  # Append the zipcode to the query
    if pd.notnull(row['phone']):  # Check if the phone number is not null
        text_query += f" {row['phone']}"  # Append the phone number to the query
    if pd.notnull(row['cuisine_description']):  # Check if the cuisine type is not null
        text_query += f" {row['cuisine_description']}"  # Append the cuisine type to the query

    # Define the query
    data = {
        'textQuery': text_query
    }

    # Make the POST request
    response = requests.post(url, headers=headers, json=data)

    # Process the response
    if response.status_code == 200:
        places_data = response.json().get('places', [])
        
        # Append the place details and reviews to the respective lists
        for place in places_data:
            display_name = place.get('displayName', {}).get('text', '').strip()
            formatted_address = place.get('formattedAddress', '').strip()
            address_first_part = formatted_address.split(',')[0].strip()

            # Check if the display name matches the first part of the formatted address
            if display_name.lower() == address_first_part.lower():
                note = 'Possible address only, no specific business match'
            else:
                note = ''
                places_results.append({
                    'camis': row['camis'],
                    'place_id': place.get('id', ''),
                    'display_name': display_name,
                    'formatted_address': formatted_address
                })

                for review in place.get('reviews', []):
                    # Extract the unique review identifier and contributor ID
                    review_id = review.get('name', '').split('/')[-1]
                    contributor_id = review.get('authorAttribution', {}).get('uri', '').split('/')[-2]
                    reviews_results.append({
                        'camis': row['camis'],
                        'place_id': place.get('id', ''),
                        'review_id': review_id,
                        'review_relative_time': review.get('relativePublishTimeDescription', ''),
                        'review_rating': review.get('rating', ''),
                        'review_text': review.get('text', {}).get('text', ''),
                        'review_language': review.get('text', {}).get('languageCode', ''),
                        'author_display_name': review.get('authorAttribution', {}).get('displayName', ''),
                        'contributor_id': contributor_id,
                        'author_photo_uri': review.get('authorAttribution', {}).get('photoUri', ''),
                        'publish_time': review.get('publishTime', '')
                    })

            # Log the API call with display_name, formatted_address, and note
            api_logs.append({
                'row_number': current_row,
                'query': text_query,
                'status_code': response.status_code,
                'display_name': display_name,
                'formatted_address': formatted_address,
                'note': note
            })
    else:
        print(f"Failed to retrieve data for {text_query}: {response.status_code}")
        api_logs.append({
            'row_number': current_row,
            'query': text_query,
            'status_code': response.status_code,
            'display_name': '',
            'formatted_address': '',
            'note': 'API call failed'
        })

    # Save progress at regular intervals
    if current_row % save_interval == 0:
        pd.DataFrame(places_results).to_csv('places_progress.csv', index=False)
        pd.DataFrame(reviews_results).to_csv('reviews_progress.csv', index=False)
        pd.DataFrame(api_logs).to_csv('api_log_progress.csv', index=False)

# Save the final results
pd.DataFrame(places_results).to_csv('places_final.csv', index=False)
pd.DataFrame(reviews_results).to_csv('reviews_final.csv', index=False)
pd.DataFrame(api_logs).to_csv('api_log_final.csv', index=False)


In [ ]:
a.main(inspections_df, g_places_api_key, save_interval=2)

In [2]:
places_df = pd.read_csv('places_final.csv')

In [3]:
places_df['url'] = "https://www.google.com/maps/place/?q=place_id:" + places_df['place_id']

places_df.head()

,camis,place_id,display_name,formatted_address,note,url
0,41549862,ChIJIyiwrqX0wokR1x1J2xWMNsg,Luciano’s Pizza,"1005 Morris Park Ave # A, Bronx, NY 10462, USA",NaN,https://www.google.com/maps/place/?q=place_id:...
1,41690495,ChIJwTC7A55ZwokRPNX9g7ngbaI,Starbucks,"219 1st Ave., New York, NY 10003, USA",NaN,https://www.google.com/maps/place/?q=place_id:...
2,50107082,ChIJOzljFpRbwokREY9_N5JAeM0,New No. 1,"1082 Bedford Ave #1, Brooklyn, NY 11216, USA",NaN,https://www.google.com/maps/place/?q=place_id:...
3,50099824,ChIJwWeC4ppbwokRpaADwmGNVtI,India House,"586 Nostrand Ave., Brooklyn, NY 11216, USA",NaN,https://www.google.com/maps/place/?q=place_id:...
4,50106021,ChIJy4EKcZ1bwokRAxycHPSTeSU,735 Nostrand Ave.,"735 Nostrand Ave., Brooklyn, NY 11216, USA","Possible address only, no specific business match",https://www.google.com/maps/place/?q=place_id:...


In [18]:
places_df.url[1]

'https://www.google.com/maps/place/?q=place_id:ChIJwTC7A55ZwokRPNX9g7ngbaI'

In [4]:
# Specify the path where you want to save the text file
output_file_path = 'urls_1.txt'

# Extract the 'url' column as a Series and convert it to a list
url_list = places_df['url'].tolist()

# Open the text file in write mode and write the URLs
with open(output_file_path, 'w') as txt_file:
    for url in url_list:
        txt_file.write(url + '\n')